In [5]:
import numpy as np
import tensorflow as tf
import numpy as np
import os

In [6]:
import pathlib
data_dir = pathlib.Path("../raw_data/data")

In [7]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

375974


In [8]:
batch_size = 512
img_height = 128
img_width = 128

In [9]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 375974 files belonging to 82 classes.
Using 300780 files for training.


2022-06-07 12:17:03.939290: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
 val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = tuple(train_ds.class_names)
print(class_names)

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
from keras.layers.core import Activation


num_classes = 82

model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(32, (2, 2),input_shape=(128, 128, 3),activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size =(2, 2)),
    
    tf.keras.layers.Conv2D(32, (2, 2),activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size =(2, 2)),
    
    tf.keras.layers.Conv2D(64, (2, 2), activation="relu", padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size =(2, 2)),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(num_classes,activation='softmax')
])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es=EarlyStopping(patience=5)

model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [ ]:
%%time


hist = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=100,
  callbacks=[es]
).history

In [ ]:
model.save("../modelDL1-MT")